In [ ]:
!pip install pandas --upgrade

In [ ]:
import pandas as pd

In [ ]:
XY_train = pd.read_pickle("XY_train.pickle")
XY_test = pd.read_pickle("XY_test.pickle")

In [ ]:
def change_types(df):    
    for col in df.columns[df.dtypes == 'int64']:
        df[col] = df[col].astype(np.int32)
    return df

XY_train = change_types(XY_train)
XY_test = change_types(XY_test)

XY_train['is_mal'] = XY_train['is_mal'].astype(np.int8)
XY_train['shop_id'] = XY_train['shop_id'].astype(np.int8)
XY_train['city'] = XY_train['city'].astype(np.int8)
XY_train['is_en_mal'] = XY_train['is_en_mal'].astype(np.int8)
XY_train['year'] = XY_train['year'].astype(np.int8)
XY_train['month'] = XY_train['month'].astype(np.int8)
XY_train['date_block_num'] = XY_train['date_block_num'].astype(np.int8)

In [ ]:
def lag_feature(df, lags, col):    
    tmp = df[['date_block_num', 'shop_id','item_id', col]]
    for i in tqdm(lags):
        gc.collect()
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] = shifted['date_block_num'] + i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

def mean_encoding(df, groupby_feats, target, enc, lags):
    print('Features: ' , groupby_feats)
    features = df[[*groupby_feats, target]]\
             .groupby(groupby_feats, as_index=False)\
             .agg(['mean'])
   
    features.columns = [enc]
    
    df = df.merge(features, on=groupby_feats, how='left')
    df[enc] = df[enc].astype(np.float16)
    df = lag_feature(df, lags, enc)
    df.drop(enc, axis=1, inplace=True)
    return df

In [ ]:
import time
ts = time.time()
X_target_encoded = XY_train
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num'], 'item_cnt_day', 'date_avg_item_cnt', [1])
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'item_id'], 
                                'item_cnt_day', 'date_item_avg_item_cnt', [1, 2, 3, 6])
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'shop_id'], 
                                 'item_cnt_day', 'date_shop_avg_item_cnt', [1,2,3,6])

X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'item_category_id'], 
                                 'item_cnt_day', 'date_cat_avg_item_cnt', [1, 6])
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'shop_id', 'item_category_id'], 
                                 'item_cnt_day', 'date_shop_cat_avg_item_cnt', [1, 6])
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'item_id', 'item_category_id'], 'item_cnt_day', 'date_shop_cat_avg_item_cnt', [1, 6])

X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'city'], 'item_cnt_day', 'date_city_avg_item_cnt', [1, 6])
X_target_encoded = mean_encoding(X_target_encoded, ['date_block_num', 'item_id', 'city'], 'item_cnt_day', 'date_item_city_avg_item_cnt', [1, 6]) 

time.time() - ts